# Import all the libraries

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
#from sklearn_genetic import GASearchCV

#from sklearn_genetic.space import Categorical, Integer, Continuous
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
#from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.cross_decomposition import PLSCanonical
from xgboost import XGBClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import *
import matplotlib.pyplot as plt
import pickle
import numpy as np
import time
#from tpot import TPOTClassifier
import shap

from sklearn.metrics import accuracy_score
#import tensorflow as tf
#from autogluon.tabular import TabularDataset, TabularPredictor
#import keras 
#from keras.losses import BinaryCrossentropy

from sklearn.ensemble import VotingClassifier

import subprocess # TO RUN VBEOSA

#from scikeras.wrappers import KerasClassifier # TO USE VOTING

from EKAN_functions import *
import statsmodels.api as sm
from sklearn.model_selection import train_test_split


from tabtransformertf.models.fttransformer import FTTransformerEncoder, FTTransformer

from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import train_test_split
import torch


# VALIDATION

##### To estimate the model performance in a proper way a nested CV may be better than a normal CV

### PIPELINE

In [ ]:

X=pd.read_csv('your_dataset_corrected_total_x.csv')
y=pd.read_csv('your_dataset_corrected_total_y.csv')
models,model_p,dataset,dataset_p,Filt_1,Filt_2,sampled_arrays=KAN_tr_m(X,y.astype('int64'),X,y.astype('int64'),k=400,random_s=True,m='KAN',fil=True,subs=False,k1=4,k2=4,N=8)


In [ ]:
model_p.plot()

In [ ]:
def KAN_pred(X_test):
    
    preds=[]


    for model,filt,subgroup in zip(models,Filt_1,sampled_arrays):


        X_test_s=X_test[filt]
        X_test_s=pd.DataFrame(X_test_s)
        dataset = {}
        dataset['test_input'] = torch.from_numpy(X_test_s.values)
        pred=torch.argmax(model(dataset['test_input']), dim=1).numpy().transpose()
        preds.append(pred)
    
    preds=preds[Filt_2[0]]
    preds=pd.DataFrame(preds)
    dataset_p= {}
    dataset_p['test_input'] = torch.from_numpy(preds.values)

    y_pred=torch.argmax(model_p(dataset_p['test_input']), dim=1).numpy().transpose()
    return y_pred



In [ ]:
explainer = shap.PermutationExplainer(KAN_pred,X)
data_used=X

shap_values = explainer.shap_values(data_used)